In [1]:
import os
cur_dir = os.getcwd()
par_dir = os.path.abspath(os.path.join(cur_dir,os.pardir))
os.chdir(par_dir)

In [4]:
import os
import numpy as np
import pandas as pd
from vae_earlystopping import EarlyStopping
import sys
import torch
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from model1.m2 import Model2
from loss1.l2 import loss2
import joblib
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from kl_divergence.cyclical import gamma_cyclical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
### 구하기
import json

### Space Time 넣기  random_state
results = {
    "R2_Space_Time": [],      # bce_binary * mse (최종)
    "R2_Except_Space_Time": [],     # bce_prob * mse (sigmoid 가중)
}
for i in np.random.randint(1,100,size = 20):
    x_data = np.load('./data/metal.npy')
    c_data = np.load('./data/pre_re_fin.npy')
    
    x_train,x_test,c_train,c_test = train_test_split(x_data,c_data, random_state = i,test_size = 0.4)
    x_val,x_test,c_val,c_test = train_test_split(x_test,c_test,random_state = i, test_size = 0.5)
    x_scaler = MinMaxScaler()
    c_scaler = MinMaxScaler()
    x_train = x_scaler.fit_transform(x_train)
    c_train = c_scaler.fit_transform(c_train)
    x_val,x_test = [x_scaler.transform(x) for x in [x_val,x_test]]
    c_val,c_test = [c_scaler.transform(c) for c in [c_val,c_test]]

    x_train,x_val,x_test = [torch.tensor(x, dtype = torch.float32) for x in [x_train,x_val,x_test]]
    c_train,c_val,c_test = [torch.tensor(c, dtype = torch.float32) for c in [c_train,c_val,c_test]]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_data = [x_train,x_train,c_train]
    val_data = [x_val,x_val,c_val]
    test_data = [x_test,x_test,c_test]
    train_data = TensorDataset(*train_data)
    val_data = TensorDataset(*val_data)
    test_data = TensorDataset(*test_data)
    datas = [train_data,val_data,test_data]
    train_loader,val_loader,test_loader = [DataLoader(x,batch_size = 64,shuffle=False) for x in datas]
    x_sample,x2_sample, c_sample= next(iter(train_loader))

    x_dim = x_sample.shape[1]
    x2_dim= x2_sample.shape[1]
    c_dim = c_sample.shape[1]

    model = Model2(x_dim,x2_dim,c_dim, z_dim=8,z2_dim = 8).to(device)
    early_stopping = EarlyStopping(patience=40,min_delta = 1e-9)
    optimizer = optim.Adam(model.parameters(),lr = 1e-3, weight_decay=1e-5)
    epochs = 800
    gamma = gamma_cyclical(800, gamma_max=0.1, cycle_len=200,ratio=0.5)
    for epoch in range(1,epochs+1):
        model.train()
        t_loss= 0
        for x,x2, c in train_loader:
            x,x2,c = x.to(device),x2.to(device),c.to(device)
            optimizer.zero_grad()
            bce_logit,x_hat, mu, logvar,mu2,logvar2 = model(x,x2,c)
            loss_dict = loss2(bce_logit,x_hat,x,x2,mu,logvar,mu2,logvar2,a=1,b=1,c=gamma,d=gamma)
            loss_dict['loss'].backward()
            optimizer.step()
            t_loss +=loss_dict['loss'].item()
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for v_x,v2_x, v_c in val_loader:
                v_x,v2_x,v_c = v_x.to(device),v2_x.to(device),v_c.to(device)
                v_bce_logit,v_x_hat, v_mu, v_logvar,v2_mu,v2_logvar = model(v_x,v2_x,v_c)
                loss_dict = loss2(v_bce_logit,v_x_hat, v_x,v2_x, v_mu,v_logvar,v2_mu,v2_logvar,a=1,b=1,c=gamma,d=gamma)
                v_loss += loss_dict['loss'].item()
        avg_train_loss = t_loss/len(train_loader)
        avg_val_loss = v_loss/len(val_loader)

        if epoch % 20 ==0  or epoch ==2:
            print(f'Epoch [{epoch}/{epochs}]|Train:{avg_train_loss:.4f} |Val:{avg_val_loss:.4f}')
        if early_stopping(avg_val_loss,model):
            break

    early_stopping.load_best_model(model)
    model.eval()
    all_x_hat = []
    all_x_true = []
    mse_logit_list = []
    x_true_list = []
    with torch.no_grad():
        for x_t,x2_t, c_t in test_loader:
            x_t,x2_t,c_t = x_t.to(device),x2_t.to(device),c_t.to(device)
            bce_logit_t,x_hat, mu_t,logvar_t,mu2_t,logvar2_t = model(x_t,x2_t,c_t)
            x_true = (x_t>0).float()
            mse_logit_list.append(x_hat.cpu().numpy())
            x_true_list.append(x_t.cpu().numpy())
            all_x_hat.append(bce_logit_t.detach().cpu())
            all_x_true.append(x_true.detach().cpu())
    all_x_hat_1 = torch.cat([a.flatten() for a in all_x_hat])
    all_x_true = torch.cat([a.flatten() for a in all_x_true])
    mse_logits = np.vstack(mse_logit_list)
    x_true = np.vstack(x_true_list)
    ### BCE-> 0,1 표현하여 MSE에 곱하기
    all_x_hat_tensor = torch.cat(all_x_hat, dim=0) # (Total_Samples, x_dim) 형태로 결합
    bce_logits_np = all_x_hat_tensor.cpu().numpy()
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    bce_prob = sigmoid(bce_logits_np)
    bce_binary = (bce_prob >= 0.5).astype(np.float32)
    x_hat_fin = x_scaler.inverse_transform(mse_logits)
    x_true = x_scaler.inverse_transform(x_true)
    final_x_sig = x_hat_fin*bce_prob
    from sklearn.metrics import r2_score,mean_squared_error
    r2_bce_mse_sig = r2_score(x_true.flatten(),final_x_sig.flatten())
    ##r2저장
    results["R2_Space_Time"].append(float(r2_bce_mse_sig))

### Space time 제외
for i in np.random.randint(1,100,size = 20):
    x_data = np.load('./data/metal.npy')
    c_data = np.load('./data/pre_re_exc_sp.npy')
    
    x_train,x_test,c_train,c_test = train_test_split(x_data,c_data, random_state = i,test_size = 0.4)
    x_val,x_test,c_val,c_test = train_test_split(x_test,c_test,random_state = i, test_size = 0.5)
    x_scaler = MinMaxScaler()
    c_scaler = MinMaxScaler()
    x_train = x_scaler.fit_transform(x_train)
    c_train = c_scaler.fit_transform(c_train)
    x_val,x_test = [x_scaler.transform(x) for x in [x_val,x_test]]
    c_val,c_test = [c_scaler.transform(c) for c in [c_val,c_test]]

    x_train,x_val,x_test = [torch.tensor(x, dtype = torch.float32) for x in [x_train,x_val,x_test]]
    c_train,c_val,c_test = [torch.tensor(c, dtype = torch.float32) for c in [c_train,c_val,c_test]]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_data = [x_train,x_train,c_train]
    val_data = [x_val,x_val,c_val]
    test_data = [x_test,x_test,c_test]
    train_data = TensorDataset(*train_data)
    val_data = TensorDataset(*val_data)
    test_data = TensorDataset(*test_data)
    datas = [train_data,val_data,test_data]
    train_loader,val_loader,test_loader = [DataLoader(x,batch_size = 64,shuffle=False) for x in datas]
    x_sample,x2_sample, c_sample= next(iter(train_loader))

    x_dim = x_sample.shape[1]
    x2_dim= x2_sample.shape[1]
    c_dim = c_sample.shape[1]

    model = Model2(x_dim,x2_dim,c_dim, z_dim=8,z2_dim = 8).to(device)
    early_stopping = EarlyStopping(patience=40,min_delta = 1e-9)
    optimizer = optim.Adam(model.parameters(),lr = 1e-3, weight_decay=1e-5)
    epochs = 800
    gamma = gamma_cyclical(800, gamma_max=0.1, cycle_len=200,ratio=0.5)
    for epoch in range(1,epochs+1):
        model.train()
        t_loss= 0
        for x,x2, c in train_loader:
            x,x2,c = x.to(device),x2.to(device),c.to(device)
            optimizer.zero_grad()
            bce_logit,x_hat, mu, logvar,mu2,logvar2 = model(x,x2,c)
            loss_dict = loss2(bce_logit,x_hat,x,x2,mu,logvar,mu2,logvar2,a=1,b=1,c=gamma,d=gamma)
            loss_dict['loss'].backward()
            optimizer.step()
            t_loss +=loss_dict['loss'].item()
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for v_x,v2_x, v_c in val_loader:
                v_x,v2_x,v_c = v_x.to(device),v2_x.to(device),v_c.to(device)
                v_bce_logit,v_x_hat, v_mu, v_logvar,v2_mu,v2_logvar = model(v_x,v2_x,v_c)
                loss_dict = loss2(v_bce_logit,v_x_hat, v_x,v2_x, v_mu,v_logvar,v2_mu,v2_logvar,a=1,b=1,c=gamma,d=gamma)
                v_loss += loss_dict['loss'].item()
        avg_train_loss = t_loss/len(train_loader)
        avg_val_loss = v_loss/len(val_loader)

        if epoch % 20 ==0  or epoch ==2:
            print(f'Epoch [{epoch}/{epochs}]|Train:{avg_train_loss:.4f} |Val:{avg_val_loss:.4f}')
        if early_stopping(avg_val_loss,model):
            break

    early_stopping.load_best_model(model)
    model.eval()
    all_x_hat = []
    all_x_true = []
    mse_logit_list = []
    x_true_list = []
    with torch.no_grad():
        for x_t,x2_t, c_t in test_loader:
            x_t,x2_t,c_t = x_t.to(device),x2_t.to(device),c_t.to(device)
            bce_logit_t,x_hat, mu_t,logvar_t,mu2_t,logvar2_t = model(x_t,x2_t,c_t)
            x_true = (x_t>0).float()
            mse_logit_list.append(x_hat.cpu().numpy())
            x_true_list.append(x_t.cpu().numpy())
            all_x_hat.append(bce_logit_t.detach().cpu())
            all_x_true.append(x_true.detach().cpu())
    all_x_hat_1 = torch.cat([a.flatten() for a in all_x_hat])
    all_x_true = torch.cat([a.flatten() for a in all_x_true])
    mse_logits = np.vstack(mse_logit_list)
    x_true = np.vstack(x_true_list)
    ### BCE-> 0,1 표현하여 MSE에 곱하기
    all_x_hat_tensor = torch.cat(all_x_hat, dim=0) # (Total_Samples, x_dim) 형태로 결합
    bce_logits_np = all_x_hat_tensor.cpu().numpy()
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    bce_prob = sigmoid(bce_logits_np)
    bce_binary = (bce_prob >= 0.5).astype(np.float32)
    x_hat_fin = x_scaler.inverse_transform(mse_logits)
    x_true = x_scaler.inverse_transform(x_true)
    final_x_sig = x_hat_fin*bce_prob
    from sklearn.metrics import r2_score,mean_squared_error
    r2_bce_mse_sig = r2_score(x_true.flatten(),final_x_sig.flatten())
    ##각각의 r2저장
    results["R2_Except_Space_Time"].append(float(r2_bce_mse_sig))



Epoch [2/800]|Train:3.6954 |Val:3.6538
EarlyStopping counter: 1 out of 40
Epoch [20/800]|Train:1.9976 |Val:2.0950
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
Epoch [40/800]|Train:1.4272 |Val:1.5019
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
Epoch [60/800]|Train:0.9762 |Val:1.0614
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
Epoch [80/800]|Train:0.7639 |Val:0.9036
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of

C:\Users\User\AppData\Local\Temp\ipykernel_8632\3223166017.py:117: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch [2/800]|Train:3.8756 |Val:3.6129
Epoch [20/800]|Train:1.9368 |Val:1.9866
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
Epoch [40/800]|Train:1.3520 |Val:1.3898
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
Epoch [60/800]|Train:0.9301 |Val:1.0226
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
Epoch [80/800]|Train:0.7419 |Val:0.8466
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of

C:\Users\User\AppData\Local\Temp\ipykernel_8632\3223166017.py:117: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch [2/800]|Train:3.5931 |Val:3.5513
Epoch [20/800]|Train:2.1093 |Val:2.2354
Epoch [40/800]|Train:1.1207 |Val:1.1861
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
Epoch [60/800]|Train:0.8599 |Val:0.9439
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
Epoch [80/800]|Train:0.7292 |Val:0.8662
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of

C:\Users\User\AppData\Local\Temp\ipykernel_8632\3223166017.py:117: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


ValueError: Input X contains infinity or a value too large for dtype('float64').

In [3]:
results

{'R2_Space_Time': [0.8396199941635132],
 'R2_Except_Space_Time': [0.8546582460403442,
  0.8727657198905945,
  0.8557615280151367,
  0.8380854725837708,
  0.8559621572494507,
  0.8200620412826538,
  0.8386325836181641,
  0.8605333566665649,
  0.860270619392395,
  0.8594616055488586,
  0.8719536662101746,
  0.8594543933868408,
  0.8434523344039917,
  0.8431404829025269,
  0.8682723045349121,
  0.8903210759162903,
  0.8251007795333862,
  0.8589532971382141,
  0.8757970333099365,
  0.8600134253501892]}

In [ ]:

save_path = "./results_Space_Presence.json"
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("Saved:", save_path)